# SASL Preprocessing Pipeline: Frames, Resizing, and Alignment

In [ ]:

This notebook brings together multiple preprocessing steps:
1. Extract frames from video
2. Resize and normalise them
3. Align gloss annotations with frame files

### Objectives:
- Apply preprocessing in one reproducible pipeline
- Organise outputs for model training
- Save frame-gloss alignment table


In [ ]:
# Step 1: Import required libraries
import cv2
import os
import pandas as pd
from pathlib import Path


In [ ]:
# Step 2: Define frame extraction function

def extract_frames(video_path, output_folder, frame_rate=5):
    cap = cv2.VideoCapture(str(video_path))
    success, frame = cap.read()
    count, saved = 0, 0
    output_folder = Path(output_folder)
    output_folder.mkdir(parents=True, exist_ok=True)

    while success:
        if int(cap.get(1)) % frame_rate == 0:
            filename = output_folder / f"frame_{saved:04d}.jpg"
            cv2.imwrite(str(filename), frame)
            saved += 1
        success, frame = cap.read()
        count += 1
    cap.release()
    print(f"Extracted {saved} frames from {video_path.name}")


In [ ]:
# Step 3: Define resizing and normalisation function

def resize_and_normalise(image_path, output_path, size=(64, 64)):
    img = cv2.imread(str(image_path))
    if img is None:
        print(f"Error loading image: {image_path}")
        return
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img_resized = cv2.resize(img_gray, size)
    img_normalised = (img_resized / 255.0 * 255).astype('uint8')
    output_path.parent.mkdir(parents=True, exist_ok=True)
    cv2.imwrite(str(output_path), img_normalised)


In [ ]:
# Step 4: Define gloss alignment function

def align_glosses(annotation_csv, frame_dir, frame_rate=5):
    df_anno = pd.read_csv(annotation_csv)
    frame_files = sorted(Path(frame_dir).glob('*.jpg'))
    frame_times = [i / frame_rate for i in range(len(frame_files))]
    aligned = pd.DataFrame({'frame_file': [f.name for f in frame_files], 'timestamp': frame_times})
    aligned['gloss'] = None

    for _, row in df_anno.iterrows():
        idx = min(range(len(frame_times)), key=lambda i: abs(frame_times[i] - row['timestamp']))
        aligned.at[idx, 'gloss'] = row['gloss']

    aligned['gloss'].fillna(method='ffill', inplace=True)
    return aligned


In [ ]:
# Step 5: Run the full pipeline

video_path = Path('../data/videos/sample_video.mp4')
frame_dir = Path('../data/frames/sample_video')
resized_dir = Path('../data/processed_images/sample_video')
annotation_csv = Path('../data/annotations/sample_video_annotations.csv')
output_csv = Path('../data/processed_alignments/sample_video_pipeline_output.csv')

extract_frames(video_path, frame_dir, frame_rate=5)

for img_file in frame_dir.glob('*.jpg'):
    resize_and_normalise(img_file, resized_dir / img_file.name)

alignment = align_glosses(annotation_csv, frame_dir, frame_rate=5)
output_csv.parent.mkdir(exist_ok=True, parents=True)
alignment.to_csv(output_csv, index=False)

print(f"Preprocessing complete. Alignment saved to {output_csv}")
